In [98]:
import os
import sys
import pandas as pd
import openpyxl

import time
from datetime import datetime
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# NAVER BLOG Crawling
- title/url 

In [47]:
# 검색어 입력
query = input("검색어 : ")
location = str(query)

# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

print(startDate, endDate)


# Chrome driver 환경설정 및 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)
base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
driver.get(base_url)


# 총 검색 결과
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
search_number = soup.find(class_="search_number")
print("총 블로그글 개수 : ", search_number.text)


# 연관 검색어 
sub_keywords_data = soup.select('div.area_keyword > div.list')
sub_keywords = []
for i in sub_keywords_data: 
    sub_keywords.append(i.get_text())
print("연관검색어:",*sub_keywords)


# blog url/title 가져오기
total_num = search_number.text.replace("건",'').replace(",",'')
total_num = int(total_num)
end_page=total_num//7+1
print(end_page)

crawling_blog_data = [] 

try:
    for page_num in range(1,end_page):
        search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
        driver.get(search_url)
        time.sleep(0.1)
        print(f'{page_num}번째 페이지 접속')

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        print('html 가져오기 완료')

        # 블로그 제목, url, 포스팅 날짜 가져오기
        posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
        print('posts_data 가져오기 완료')
        posts_date_data = soup.select('div.info_post > div.writer_info')
        print('posts_date_data 가져오기 완료')

        for post, post_date in zip(posts_data, posts_date_data):
            title = post.text.replace('\n','').strip()
            href = post.attrs['href']
            date = post_date.find('span','date').text.strip()# Corrected this line
            try : 
                date_timestamp = datetime.strptime(date, '%Y. %m. %d.').strftime('%Y-%m-%d')
            except: 
                date_timestamp = date
            crawling_blog_data.append({"location": location, "title": title, "url": href, "posted_date": date_timestamp})
        print("리스트 담기 완료")

except Exception as e:
            print(f"error : {page_num}페이지 - {e}")

print("carwling data :",crawling_blog_data)

2024-05-16 2024-05-23
총 블로그글 개수 :  425건
연관검색어: 
회기역 맛집회기역 경의중앙선회기역 1호선회기역 카페회기역 시간표회기역 경춘선회기역 술집회기역 경의중앙선 시간표회기역 미용실회기역 놀거리회기역 오피스텔회기역 미래사주회기역 파전골목회기역 파전회기역 꽃집회기역 아파트회기역 근처 맛집회기역 물품보관함회기역 고기집청량리역


61
1번째 페이지 접속
html 가져오기 완료
posts_data 가져오기 완료
posts_date_data 가져오기 완료
리스트 담기 완료
2번째 페이지 접속
html 가져오기 완료
posts_data 가져오기 완료
posts_date_data 가져오기 완료
리스트 담기 완료
3번째 페이지 접속
html 가져오기 완료
posts_data 가져오기 완료
posts_date_data 가져오기 완료
리스트 담기 완료
4번째 페이지 접속
html 가져오기 완료
posts_data 가져오기 완료
posts_date_data 가져오기 완료
리스트 담기 완료
5번째 페이지 접속
html 가져오기 완료
posts_data 가져오기 완료
posts_date_data 가져오기 완료
리스트 담기 완료
6번째 페이지 접속
html 가져오기 완료
posts_data 가져오기 완료
posts_date_data 가져오기 완료
리스트 담기 완료
7번째 페이지 접속
html 가져오기 완료
posts_data 가져오기 완료
posts_date_data 가져오기 완료
리스트 담기 완료
8번째 페이지 접속
html 가져오기 완료
posts_data 가져오기 완료
posts_date_data 가져오기 완료
리스트 담기 완료
9번째 페이지 접속
html 가져오기 완료
posts_data 가져오기 완료
posts_date_data 가져오기 완료
리스트 담기 완료
10번째 페이지 접속
html 가져오기 완료
posts_data 가져오기 완료
posts_date_data 가져오기 완료
리스트 담기 완료
11번째 페이지 접속
html 가져오기 완료
pos

In [48]:
len(crawling_blog_data)

420

In [86]:
df = pd.DataFrame(crawling_blog_data)
df


,location,title,url,posted_date
0,회기역,회기역 카페 - 회기역 근처 가볍게 점심 먹기 괜찮은 [ 읍천리382 ] 내돈사먹 후기,https://blog.naver.com/hyoddiya/223449731139,2024-05-17
1,회기역,회기역 호텔 모우 경희대 한국외대 근처숙소,https://blog.naver.com/mukgojayo/223453331578,2024-05-21
2,회기역,"위아짐 회기역pt 전문 헬스장, 피티 초보라면 조보경트레이너!",https://blog.naver.com/orot-i/223454276784,2024-05-22
3,회기역,"[교린이QnA]회기역 대학생 치아교정, '심미선'으로 나만의 맞춤 교정을 찾으세요",https://blog.naver.com/wenly7/223449724947,2024-05-17
4,회기역,호텔 모우 회기역 호텔 가성비 좋은 비지니스호텔,https://blog.naver.com/jems80/223449271410,2024-05-17
...,...,...,...,...
415,회기역,[나도 이런 창업해야지] LUCKYMEAL_럭키밀 어플 내돈내산 추천 ♡,https://blog.naver.com/mean0e/223448905481,2024-05-16
416,회기역,회기 위험한맛집 : 경성고기꾼 내돈내산,https://blog.naver.com/laboratory_man/22344775...,2024-05-16
417,회기역,[123] 회기역 역명판 <2024년>,https://blog.naver.com/minju4158/223452893026,2024-05-20
418,회기역,회기역 빵맛집 베이커리카페 964커피&너만보여크루아상,https://blog.naver.com/whao33/223449041413,2024-05-16


In [90]:
df1 = df.query('posted_date.str.contains("시간 전")')
df1.sort_values('posted_date', axis=0, ascending=False, inplace=True) 
df1

C:\Users\20061\AppData\Local\Temp\ipykernel_32952\640770367.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.sort_values('posted_date', axis=0, ascending=False, inplace=True)


,location,title,url,posted_date
203,회기역,회기) 이모네포장마차,https://blog.naver.com/klain_12/223455440100,9시간 전
325,회기역,02. 2024 _ 나 진짜 이런 정신머리로 이렇게 간다고?,https://blog.naver.com/princess_99_/223455448712,9시간 전
419,회기역,단순하게 커피만 마시는 카페는 No! 보드게임까지 즐기는 레이지클라우드 카페가...,https://blog.naver.com/alicehouse31/223453687980,8시간 전
81,회기역,동대문구 회기역 삼육보건대학교 VIP식당 고급도시락배달,https://blog.naver.com/brotherdosirak_nw/22344...,8시간 전
230,회기역,상봉아이폰수리 신속하게 해결돼요,https://blog.naver.com/sheepgh8186/223454866890,8시간 전
...,...,...,...,...
202,회기역,MJ스포랜드 회기점 위치 및 정보,https://blog.naver.com/usefullbox/223455331524,16시간 전
280,회기역,2024년 5월 3주차,https://blog.naver.com/moomoodeath/223455319192,16시간 전
40,회기역,회기역 맛집 회기왕갈비탕 점심 혼밥하기 좋은 외대 맛집,https://blog.naver.com/kwakyoukyeong/223455354721,16시간 전
209,회기역,류아 필라테스 회기점 위치 및 정보,https://blog.naver.com/usefullbox/223455333236,16시간 전


In [93]:
df2 = df.query('not posted_date.str.contains("시간 전")')
df2.sort_values('posted_date', axis=0, ascending=False, inplace=True) 
df2

C:\Users\20061\AppData\Local\Temp\ipykernel_32952\182116724.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.sort_values('posted_date', axis=0, ascending=False, inplace=True)


,location,title,url,posted_date
129,회기역,경희대맛집 분위기맛집 마라카츠 모짜렐라치즈카츠 마제소바 오레노카츠 경희대점,https://blog.naver.com/st338/223455923578,44분 전
105,회기역,아차산 ~ 망우산,https://blog.naver.com/blessingsuk/223455927185,42분 전
212,회기역,"대전, 서울 하이거, BYD 운행노선현황",https://blog.naver.com/angel060711/223454408552,2024-05-22
34,회기역,회기역 가성비 퀄리티 오마카세 스시히노하루 런치 내돈내산 후기,https://blog.naver.com/a1787a1787/223452526054,2024-05-22
408,회기역,[외대필라테스/이문동필라테스] 두피건강에 좋은 음식_옐로우필라테스,https://blog.naver.com/yellowpilates1/22345445...,2024-05-22
...,...,...,...,...
335,회기역,동대문구 실용음악학원 동대문구 드럼학원 동대문구 보컬학원 동대문구 댄스학원...,https://blog.naver.com/fwinds/223448265584,2024-05-16
333,회기역,24년 4월 일기(부동산 임장/초보 체험단/ADSP 독학/프리미엄 막걸리),https://blog.naver.com/nongnogne/223444593219,2024-05-16
258,회기역,경희대카페 스터디하기좋은 대형카페 _ 8번가 8street,https://blog.naver.com/mydreameruza/223448746614,2024-05-16
260,회기역,동대문미용실 악성곱슬 볼륨매직 예스헤어 만족합니다,https://blog.naver.com/co1823/223447979677,2024-05-16


In [96]:
total_df = pd.concat([df1,df2], ignore_index=True)
total_df

,location,title,url,posted_date
0,회기역,회기) 이모네포장마차,https://blog.naver.com/klain_12/223455440100,9시간 전
1,회기역,02. 2024 _ 나 진짜 이런 정신머리로 이렇게 간다고?,https://blog.naver.com/princess_99_/223455448712,9시간 전
2,회기역,단순하게 커피만 마시는 카페는 No! 보드게임까지 즐기는 레이지클라우드 카페가...,https://blog.naver.com/alicehouse31/223453687980,8시간 전
3,회기역,동대문구 회기역 삼육보건대학교 VIP식당 고급도시락배달,https://blog.naver.com/brotherdosirak_nw/22344...,8시간 전
4,회기역,상봉아이폰수리 신속하게 해결돼요,https://blog.naver.com/sheepgh8186/223454866890,8시간 전
...,...,...,...,...
415,회기역,동대문구 실용음악학원 동대문구 드럼학원 동대문구 보컬학원 동대문구 댄스학원...,https://blog.naver.com/fwinds/223448265584,2024-05-16
416,회기역,24년 4월 일기(부동산 임장/초보 체험단/ADSP 독학/프리미엄 막걸리),https://blog.naver.com/nongnogne/223444593219,2024-05-16
417,회기역,경희대카페 스터디하기좋은 대형카페 _ 8번가 8street,https://blog.naver.com/mydreameruza/223448746614,2024-05-16
418,회기역,동대문미용실 악성곱슬 볼륨매직 예스헤어 만족합니다,https://blog.naver.com/co1823/223447979677,2024-05-16


In [100]:
file_name = f"{query}_{datetime.now().strftime('%Y%m%d')}.csv"
total_df.to_csv(file_name, index=False)

In [ ]:
df = pd.DataFrame(crawling_blog_data)
df


,location,title,url,posted_date
0,회기역,회기역 카페 - 회기역 근처 가볍게 점심 먹기 괜찮은 [ 읍천리382 ] 내돈사먹 후기,https://blog.naver.com/hyoddiya/223449731139,2024-05-17
1,회기역,회기역 호텔 모우 경희대 한국외대 근처숙소,https://blog.naver.com/mukgojayo/223453331578,2024-05-21
2,회기역,"위아짐 회기역pt 전문 헬스장, 피티 초보라면 조보경트레이너!",https://blog.naver.com/orot-i/223454276784,2024-05-22
3,회기역,"[교린이QnA]회기역 대학생 치아교정, '심미선'으로 나만의 맞춤 교정을 찾으세요",https://blog.naver.com/wenly7/223449724947,2024-05-17
4,회기역,호텔 모우 회기역 호텔 가성비 좋은 비지니스호텔,https://blog.naver.com/jems80/223449271410,2024-05-17
...,...,...,...,...
415,회기역,[나도 이런 창업해야지] LUCKYMEAL_럭키밀 어플 내돈내산 추천 ♡,https://blog.naver.com/mean0e/223448905481,2024-05-16
416,회기역,회기 위험한맛집 : 경성고기꾼 내돈내산,https://blog.naver.com/laboratory_man/22344775...,2024-05-16
417,회기역,[123] 회기역 역명판 <2024년>,https://blog.naver.com/minju4158/223452893026,2024-05-20
418,회기역,회기역 빵맛집 베이커리카페 964커피&너만보여크루아상,https://blog.naver.com/whao33/223449041413,2024-05-16
